<a href="https://colab.research.google.com/github/dilipan10/AI-DRIVEN-CRACKING-THE-MARKET-CODE-/blob/main/cracking_the_code_using_ai_driven_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries (only run once)
!pip install yfinance
!pip install tensorflow
!pip install mplfinance
!pip install seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 783.8 kB/s eta 0:00:00


In [ ]:

# Step 1: Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import seaborn as sns